In [ ]:
from mastsel import *

In [ ]:
# PSD wind/vibration: the one for MAVIS (forest of peaks)

# Directions for total jitter residual estimation (cartesian in arcsec): 
# [0,0], [5,-5], [5,5], [-5,5], [-5,-5], [15,-15], [15,15], [-15,15], [-15,-15]
# [-15, 15]x[-15, 15] , 4 punti 
# griglia fitta: ogni 3 mas 

cartPointingCoords = np.asarray([[5,5]])
NGS_flux = [10000, 30000, 5000]
NGS_SR_1650 = [0.4, 0.2, 0.6]
NGS_FWHM_mas = [90, 110, 85]
polarNGSCoords = np.asarray([[30.0,0.0], [50.0,100.0],[10.0,240.0]])
cartNGSCoords = np.asarray([polarToCartesian(polarNGSCoords[0]), polarToCartesian(polarNGSCoords[1]), polarToCartesian(polarNGSCoords[2])])
mLO = MavisLO("../data/ini", 'mavisParamsTests', 'data/windpsd_mavis.fits')
Ctot = mLO.computeTotalResidualMatrix(cartPointingCoords, cartNGSCoords, NGS_flux, NGS_SR_1650, NGS_FWHM_mas)
print('Ctot', Ctot.shape, Ctot)

In [ ]:
theta = sp.symbols('theta')
sigma_1 = sp.symbols('sigma^2_1')
sigma_2 = sp.symbols('sigma^2_2')
sigma_X = sp.symbols('sigma^2_X')
sigma_Y = sp.symbols('sigma^2_Y')
sigma_XY = sp.symbols('sigma^2_XY')
eq1 = sp.Eq(theta, sp.S(1)/sp.S(2) * sp.atan( 2*sigma_XY / ( sigma_X-sigma_Y )) )
eq2 = sp.Eq(sigma_1, sp.S(1)/sp.S(2) * ( 2*sigma_XY / sp.sin(2*theta) + sigma_X + sigma_Y ) )
eq3 = sp.Eq(sigma_2, sigma_X+sigma_Y-sigma_1 )
display(eq1)
display(eq2)
display(eq3)
matCovTT = sp.MatrixSymbol('C_TT', 2, 2)
matCovTT_rhs = sp.Matrix([[sigma_X, sigma_XY],[sigma_XY, sigma_Y]])
ee = sp.Eq(matCovTT, matCovTT_rhs)
display(ee)

In [ ]:
matR = sp.MatrixSymbol('R', 2, 6)
matP = sp.MatrixSymbol('P_alpha', 2, 5)
matW = sp.MatrixSymbol('W', 5, 6)
display(sp.Eq(matR, sp.MatMul(matP,matW)) )
matC1 = sp.MatrixSymbol('C_1', 2, 2)
matCaa = sp.MatrixSymbol('C_aa', 2, 2)
matCas = sp.MatrixSymbol('C_as', 2, 6)
matCss = sp.MatrixSymbol('C_ss', 6, 6)
matC2 = sp.MatrixSymbol('C_2', 2, 2)
display(sp.Eq(matC2,  matCaa - sp.MatMul(matCas, matR.T) - sp.MatMul(matR,matCas.T) + sp.MatMul(matR , matCss , matR.T)) )
matCnn = sp.MatrixSymbol('C_nn', 6, 6)
matC3 = sp.MatrixSymbol('C_3', 2, 2)
display(sp.Eq(matC3,  sp.MatMul(matR, matCnn, matR.T)) )
matC = sp.MatrixSymbol('C', 2, 2)
display(sp.Eq(matC,  matC1+matC2+matC3 ) )

In [ ]:
## The files in the folder are examples of data used or obtained in the jitter computation code. The parameters are the ones defined in params4jittercode.txt, unless otherwise specified. The files are the following:
#- example_gaussian_psf.fits: Gaussian PSF for the 2nd star, normalized so that a PSF with SR of 1 would have had a flux of 1 e-, i. e. it takes into account the loss of flux in the PSF core due to SR < 1 and FWHM > diffraction.
#- example_slope_noise.fits: Slope standard deviation (in mas) and bias factor in the case of the 2nd star sensed at 500 Hz.: due valori sigma e mu/shift
#- windpsd_mavis.fits: PSD of tip and tilt (in nm2/Hz) due to wind/vibrations for MAVIS. Format: [nfreq,3]. 1st index: frequency in Hz. 2nd index: Tip PSD. 3rd index: Tilt PSD.
#- minim_wind_noise12071nm2_psdmavis.fits: Table of residual variance on tip and tilt (in nm2) due to wind/vibrations for all the considered gains of the double integrator. Computed for the 2nd star sensed at 500 Hz. The noise level of 12071 nm2 corresponds to the slope error in example_slope_noise.fits. Format: [ngains,ngains,2].
#- psd_turb.fits: PSD of turbulent tip and tilt in nm2/Hz. Format: [nfreq,3]. 1st index: frequency in Hz. 2nd index: Tip PSD. 3rd index: Tilt PSD.
#- res_noise_ngs.fits: Residual variance of tip and tilt (in nm2) for all gains explored in the noise computation. Computed for the 2nd star at 500 Hz.
#- Cn.fits: Noise covariance matrix for all stars sensed at 500 Hz.
#- rec_tomo.fits: Tomographic reconstructor.
#- cov_tomo_onaxis.fits: Resulting covariance matrix of tip/tilt on axis for the tomography contribution. : C3
#- cov_noise_onaxis.fits: Resulting covariance matrix of tip/tilt on axis for the noise contribution. Computed using Cn.fits. : C3
# ok sarebbe le 9 C per le 9 direzioni specificate nei parametri
#- jitter_map.fits: Resulting jitter map for the configuration described in params4jittercode.txt. Format: [ndirections,5]. 1st & 2nd indexes: X, Y coordinates in arcsec. 3rd & 4th indexes: tip/tilt standard deviation along the major & minor axes (in mas). 5th index: Angle between the major axis and the X axis in degrees (counterclockwise).
filenames = ['Cn.fits', 'example_gaussian_psf.fits', 'psd_turb.fits',  'res_noise_ngs.fits', 'cov_noise_onaxis.fits',
             'cov_tomo_onaxis.fits', 'jitter_map.fits', 'rec_tomo.fits', 'example_slope_noise.fits']
# torna simile se moltiplico tutte le posizioni delle stelle per 2
filenames1 = ['rec_tomo.fits']
# torna simile se moltiplico tutte le posizioni delle stelle per 2
filenames2 = ['res_noise_ngs.fits']
# 'windpsd_maory.fits', 'minim_wind_noise12071nm2_psdmaory.fits',
# 'windpsd_mavis.fits', 'minim_wind_noise12071nm2_psdmavis.fits',
for name in filenames1:
    hdul = fits.open('../data/' + name)
    _data = np.asarray(hdul[0].data, np.float64)
    print(_data.shape)
#    if len(_data.shape) == 2:
#        plt.figure(figsize=(5, 5))
#        plt.subplot(1, 1, 1)
#        plt.imshow(_data, origin='lower', interpolation='nearest')
    #if len(_data.shape) == 1:
    print(_data)
#    plt.yscale('log')
#    plt.plot(_data[0]) 
#    plt.plot(_data[1]) 